In [ ]:
import torch
print(torch.__version__)

2.5.1+cu121


In [ ]:
pip install torchinfo

In [ ]:
pip install tsai

In [ ]:
from tsai.all import *

In [ ]:

print("Script started")

import os
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tsai.tsai.models.TST import TST  # Import the TST model
from tsai.tsai.models.InceptionTime import InceptionTime  # Import the InceptionTime model
from tsai.tsai.models.RNN import GRU  # Import the GRU model from tsai


import argparse
#import argparse

folder ='/fast/ocoser/DatiProva/DebuggEMG'
contents = os.listdir(folder)
print(contents)

def load_data():
    Lista_Dati = []
    Lista_label = []
    Lista_Soggetto=[]
    file_list = os.listdir(".")
    for filename in contents:
        if "Stair" in filename or "Treadmill" in filename:
            file_path = os.path.join(folder, filename)
            f = open(file_path, "r")
            print(filename)
            for row in f.readlines():
                Row_S = row.strip().split(",")
                #print(Row_S)
                # print(len(Row_S))
                Lista_Dati.append(
                    (float(Row_S[1]), float(Row_S[2]), float(Row_S[3]), float(Row_S[4]), float(Row_S[5]),
                     float(Row_S[6]),
                     float(Row_S[7]), float(Row_S[8]), float(Row_S[9]), float(Row_S[10]),
                     float(Row_S[11])))
                Lista_label.append(Row_S[12])
                #print(row)
                if len(Row_S)>13:
                    Lista_Soggetto.append(Row_S[13])
                else:
                    Lista_Soggetto.append("none")
                    #print("Sono qui",Lista_Soggetto)

    return Lista_Dati, Lista_label, Lista_Soggetto

#load_data()


def Divido_per_soggetto(Lista_dati, Lista_label, Lista_Soggetto,soggetto):
    Training =[]
    Test = []
    Label_Training = []
    Label_Test = []
    for i in range(len(Lista_Soggetto)):
        if Lista_Soggetto[i] == soggetto:
            Test.append(Lista_dati[i])
            Label_Test.append(Lista_label[i])
        elif Lista_Soggetto[i] == "none":
            continue
        else:
            Training.append(Lista_dati[i])
            Label_Training.append(Lista_label[i])

    return Training, Test, Label_Training, Label_Test




def finestre_temporali(Lista_dati, Lista_label):
    Grouped_List = []
    Lista_label_ridotta = []
    for i in range(0, len(Lista_dati), 500):
        group = Lista_dati[i:i + 500]
        group2 = Lista_label[i:i + 500]
        Grouped_List.append(group)
        Lista_label_ridotta.append(group2)

    Lista_indici = []
    Lista_dati_Corretta = []
    for i in range(len(Lista_label_ridotta)):
        #print(Lista_label_ridotta[i])
        if len(set(Lista_label_ridotta[i])) < 2:
            Lista_indici.append(set(Lista_label_ridotta[i]))
            Lista_dati_Corretta.append(Grouped_List[i])

    print("----")
   # print(len(Lista_indici))
   # print(len(Lista_dati_Corretta))
    Lista_Indici_Finale = []
    #print(Lista_indici)
    #print("So qua",Lista_indici)
    for el in Lista_indici:
        lista = list(el)
        for el2 in lista:
            Lista_Indici_Finale.append(el2)
   # print("Sono qua",len(Lista_Indici_Finale), Lista_Indici_Finale)
    #print("Sono qua",len(Lista_dati_Corretta), Lista_dati_Corretta)

    return Lista_dati_Corretta, Lista_Indici_Finale


#print("ciao")
#print(len(X2),len(X2_label))
#print(len(Y2),len(Y2_label))


## Senza Label ##
def Tolgo_Idle(X_1,y_1):
    X_new = []
    y_new = []
    for i in range(len(y_1)):
        if y_1[i] == "idle":
            continue
        else:
            X_new.append(X_1[i])
            y_new.append(y_1[i])

    return X_new, y_new




#print("****")
#print(len(X2_new), len(X2_label_new))
#print(len(Y2_new), len(Y2_label_new))

def do_label_Corretta(Nuove):
    Y_new2 = []
    for el in Nuove:
        if el =="walk-stairascent" or el =="stairascent" or el =="stairascent-walk":
            Y_new2.append(0)
        elif el == "walk-stairdescent" or el =="stairdescent" or el =="stairdescent-walk":
            Y_new2.append(1)
        elif el == "walk-rampascent" or el =="rampascent" or el =="rampascent-walk":
            Y_new2.append(2)
        elif el == "walk-rampdescent" or el =="rampdescent" or el =="rampdescent-walk":
            Y_new2.append(3)
        else:
            Y_new2.append(4)

    return Y_new2

#print(len(Train_label),Train_label)
#print(len(Test_label),Test_label)
def creo_numpy_array(Y_new2C, X_new):
    New_X = []
    New_y = []
    for i in range(len(X_new)):
        if len(X_new[i]) < 500:
            continue
        else:
            New_X.append(X_new[i])
            New_y.append(Y_new2C[i])

    New_X = np.array(New_X)
    New_y = np.array(New_y)
    return New_X, New_y




def main(soggetto):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Load and preprocess data (assuming these functions are defined)
    Lista_dati, Lista_label, Lista_Soggetto = load_data()
    print(len(Lista_dati), len(Lista_label))
    X, Y, x_label, y_label = Divido_per_soggetto(Lista_dati, Lista_label, Lista_Soggetto, soggetto)
    X2, X2_label = finestre_temporali(X, x_label)
    Y2, Y2_label = finestre_temporali(Y, y_label)
    X2_new, X2_label_new = Tolgo_Idle(X2, X2_label)
    Y2_new, Y2_label_new = Tolgo_Idle(Y2, Y2_label)
    Train_label = do_label_Corretta(X2_label_new)
    Test_label = do_label_Corretta(Y2_label_new)
    Dati_X, label_x = creo_numpy_array(Train_label, X2_new)
    Dati_test_y, label_test_y = creo_numpy_array(Test_label, Y2_new)
    print(len(Dati_test_y))

    # Convert numpy arrays to torch tensors.
    # IMPORTANT: TST expects input shape (batch, features, seq_len).
    # If Dati_X is of shape (num_samples, seq_len, features), we need to permute.
    Dati_X_torch = torch.tensor(Dati_X, dtype=torch.float32)  # shape: (N, seq_len, features)
    label_x_torch = torch.tensor(label_x, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1)

    Dati_test_y_torch = torch.tensor(Dati_test_y, dtype=torch.float32)
    label_test_y_torch = torch.tensor(label_test_y, dtype=torch.float32).unsqueeze(1)

    # Permute from (N, seq_len, features) to (N, features, seq_len)
    Dati_X_torch = Dati_X_torch.permute(0, 2, 1)
    Dati_test_y_torch = Dati_test_y_torch.permute(0, 2, 1)

    # Create DataLoaders for training and testing
    train_dataset = TensorDataset(Dati_X_torch, label_x_torch)
    test_dataset = TensorDataset(Dati_test_y_torch, label_test_y_torch)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, num_workers=2)

    # Determine input dimensions from your data
    c_in = Dati_X_torch.shape[1]   # number of features
    seq_len = Dati_X_torch.shape[2]  # sequence length
    print(seq_len)
    # Initialize the TST model
    model = GRU(
    c_in=c_in,        # Number of input features
    c_out=1,          # Regression output (1D)
    hidden_size=128,  # Hidden layer size for GRU
    n_layers=1,       # Number of GRU layers
    bidirectional=False, # Use False for standard GRU
    fc_dropout=0.2    # Dropout to prevent overfitting
    ).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop for 1 epoch
    print("Starting model training for 1 epoch...")
    model.train()
    for epoch in range(1):  # Train for 1 epoch
        print(f"Epoch {epoch+1} started...")  # Debugging print
        for batch_idx, (batch_data, batch_labels) in enumerate(train_loader):
            print(f"Processing batch {batch_idx+1}/{len(train_loader)}")  # Debugging print
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} completed.")


    # Evaluation
    model.eval()
    total_loss = 0.0
    preds = []
    truths = []
    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)  # Move batch to device
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            total_loss += loss.item() * batch_data.size(0)
            preds.append(outputs)
            truths.append(batch_labels)
    total_loss /= len(test_dataset)
    preds = torch.cat(preds, dim=0)
    truths = torch.cat(truths, dim=0)

    # Compute MAE, MSE and R2 Score
    mae = torch.mean(torch.abs(preds - truths)).item()
    mse = torch.mean((preds - truths)**2).item()
    ss_res = torch.sum((truths - preds) ** 2)
    ss_tot = torch.sum((truths - torch.mean(truths)) ** 2)
    r2 = 1 - ss_res / ss_tot

    print(f'MSE: {mse:.8f}, MAE: {mae:.8f}, R2 Score: {r2:.8f}')
    print(f"Subject processed: {soggetto}")
#main(soggetto="AB06")

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("soggetto", type=str, help="Subject identifier (e.g., AB06)")
    args = parser.parse_args()

    main(args.soggetto)  # Pass parsed argument to main


In [ ]:
from tsai.inference import load_learner

mv_clf = load_learner("models/mv_clf.pkl")
probas, target, preds = mv_clf.get_X_preds(X[splits[1]], y[splits[1]])